---

**License:** This notebook is part of WICCA and is licensed under the [GNU GPL v3.0](./LICENSE).
Copyright © 2025 Andrii Lesniak.

---

# WICCA: Wavelet-based Image Compression & Classification Analysis

WICCA is a research framework exploring the impact of wavelet compression on image classification performance.
The current goal is to assess whether reducing image size via Haar wavelet transformation retains enough information for accurate classification.

### Objectives
- Apply Haar wavelet compression to large images (>2K resolution).
- Evaluate classification performance on both original and compressed images.
- Compare results across multiple pre-trained models.


#### Set TF to use CPU globally,
My device is quite old and doesn't support CUDA (anymore). I decided that it would be easier to just ignore GPU.
<b>Run this snippet only if you got CUDA compatibility issues.</b>

In [ ]:
# import os
# # Hide GPUs from TF
# os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
#
# import tensorflow as tf
# tf.config.set_visible_devices([], 'GPU')
#
# # Now everything defaults to CPU:0
# print(tf.config.list_physical_devices("GPU"))
# print("GPUs visible to TF:", tf.config.list_physical_devices("GPU"))

## Imports

In [ ]:
import cv2
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow.keras.applications as apps
from pathlib import Path

import wicca.visualization as viz
import wicca.result_manager as rmgr
from wicca.data_loader import load_image, load_models
from wicca.wavelet_coder import HaarCoder
from wicca.classifying_tools import ClassifierProcessor
from wicca.config.constants import SIM_CLASSES_PERC, SIM_BEST_CLASS, RESULTS_FOLDER

%matplotlib inline
pd.set_option('display.float_format', '{:.5f}'.format)
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # 0=DEBUG, 1=INFO, 2=WARNING, 3=ERROR

## Models
Let's load our models using [keras.apps](https://keras.io/api/applications/).

To load image classification models, provide a dictionary that maps model names to their classes
- Keys: Descriptive model names (strings)
- Values: Either a model class reference OR a tuple of (model class, configuration dict)
- Use tuples for models requiring non-standard input shapes (e.g., 299×299 instead of 224×224)


In [ ]:
models_dict = {
    # Mobile Networks (standard 224×224)
    'MobileNetV2': apps.mobilenet_v2.MobileNetV2,

    # VGG family (standard 224×224)
    'VGG16': apps.vgg16.VGG16,
    'VGG19': apps.vgg19.VGG19,

    # ResNet family (standard 224×224)
    'ResNet50': apps.resnet50.ResNet50,
    'ResNet101': apps.resnet.ResNet101,

    # DenseNet family (standard 224×224)
    'DenseNet169': apps.densenet.DenseNet169,
    'DenseNet201': apps.densenet.DenseNet201,

    # NasNet family (custom shapes)
    'NASNetMobile': apps.nasnet.NASNetMobile,  # Uses 224×224
    'NASNetLarge': (apps.nasnet.NASNetLarge, {'shape': (331, 331)}),

    # Inception family (custom shapes)
    'InceptionV3': (apps.inception_v3.InceptionV3, {'shape': (299, 299)}),
    'InceptionResNetV2': (apps.inception_resnet_v2.InceptionResNetV2, {'shape': (299, 299)}),

    # EfficientNet family (custom shapes)
    'EfficientNetB0': apps.efficientnet.EfficientNetB0,
    'EfficientNetB1': (apps.efficientnet.EfficientNetB1, {'shape': (240, 240)}),

    # Xception (custom shape)
    'Xception': (apps.xception.Xception, {'shape': (299, 299)}),
}

models_dict_light = {
    # Mobile Networks (standard 224×224)
    'MobileNetV2': apps.mobilenet_v2.MobileNetV2,
    # VGG family (standard 224×224)
    'VGG16': apps.vgg16.VGG16,
    # EfficientNet family (custom shapes)
    'EfficientNetB1': (apps.efficientnet.EfficientNetB1, {'shape': (240, 240)}),
    # Xception (custom shape)
    'Xception': (apps.xception.Xception, {'shape': (299, 299)}),
}

classifiers = load_models(models_dict)

### Data load

Sample image

In [ ]:
path = Path('data/originals')
list_dir = [f.name for f in path.iterdir()]

idx = list_dir[4]
sample = load_image(f'{path}/{idx}')

plt.imshow(sample)
plt.axis('off')
plt.title(f'{idx}, shape: {sample.shape}')
plt.show()

## Analysis

To start out analysis we need to define a processor.
The `ClassifierProcessor` is designed to handle large-image datasets efficiently. It can process images in parallel, save results automatically, and provide visualization tools to compare original images with their classified icons.
The class takes care of all preprocessing required by each model, manages the classification workflow, and organizes results in a structured format for easy analysis. After processing, you'll find detailed classification results in your specified results folder, ready for further analysis or visualization.


There is some validation for depth input, but it is highly recommended to pass only in specified below formats.

**Please be aware, that process might take quite a while for large datasets.**

ClassifierProcessor supports a variety of depth inputs

In [ ]:
depth = 5
depth_tuple = (3, 4)
depth_range = range(2,7)
depth_list = [2, 6]
depth_tuple_list = [(3, 4), (4, 5), (5, 6)] # would fail

By default, it tries to save results into the "results" folder in the project core folder. You can change this folder by defining a new path using pathlib or just providing it as str, and passing it as an argument

In [ ]:
data_folder = 'data/originals'

res_folder_path = Path('C:/101AwesomeProject/results')
res_folder_str = 'C:/101AwesomeProject/results'

res_default = RESULTS_FOLDER

## Single depth

In [ ]:
processor = ClassifierProcessor(
    data_folder='data/originals',  # Set a directory containing images to process
    wavelet_coder=HaarCoder(),  # Set our wavelet
    transform_depth=range(1,6),  # Set the depth of transforming; accepts int | range | list
    top_classes=5,  # Set top classes for comparison
    interpolation=cv2.INTER_AREA,  # Set an interpolation method
    results_folder='results',  # Set the results folder
    log_info=True,  # Print input-related info; enabled by default
    batch_size=30 # Set size of image batch for classifier
)

General use of this framework to classify a batch of classifiers:

In [ ]:
processor.process_classifiers(classifiers, timeout=3600)

Though, you can just pass single a dict with classifier

In [ ]:
# single_classifier = load_models(single_model)
# processor.process_classifiers(single_classifier)

Additionally, you could access single model from already loaded. However, some rules should be followed.
You should use following format

`process_single_classifier("model_name_you_specified", classfiers_dict["model_name_you_specified"], timeout) `

In [ ]:
# processor.process_single_classifier("MobileNetV2", classifiers["MobileNetV2"], timeout=3600)

 ### Results presentation
Let's have a look at our results

MobileNetV2

In [ ]:
rmgr.load_summary_results(
    results_folder=res_default,
    classifier_name='MobileNetV2',
    depth=5
)                           # We can call results either by loading csv or
# results['MobileNetV2']    # By directly looking into "results" dataframe

EfficientNetB0

In [ ]:
rmgr.load_summary_results(
    results_folder=res_default,
    classifier_name='EfficientNetB0',
    depth = 5
)
# results['VGG16']

#### Compare classifiers by single depth

Let's extract values for some visualization

In [ ]:
comparison = rmgr.compare_summaries(res_default, classifiers, 5, 'mean')
names, similar_classes_pct = rmgr.extract_from_comparison(comparison, 'similar classes (%)')
names, similar_best_class = rmgr.extract_from_comparison(comparison, 'similar best class')
# similar_best_class = similar_best_class * 100

### Similar classes

In [ ]:
viz.plot_metric_radar(names, similar_classes_pct, 'Best 5 Classes Similarity', min_value=75, max_value=95)

### The best class similarity

In [ ]:
viz.plot_metric_radar(names, similar_best_class, "Best Class Similarity", min_value=75, max_value=95)

In [ ]:
viz.plot_compare_metrics(names, similar_classes_pct, similar_best_class)

##  Results comparison for different depths
In this block we compare classifying performance by multiple classifiers

Let's visualize how different depth of transformation affect our images.

In [ ]:
viz.show_image_vs_icon(sample, range(1,6), HaarCoder())

In [ ]:
viz.show_icon_on_image(
    image=sample,
    depth_value=range(1,6),
    coder=HaarCoder(),
    border_width=5,
    border_color=(0, 0, 0),
    figsize=(12,7)
)

A heatmap for visualization similar classes cases by models

In [ ]:
x = rmgr.compare_summaries(res_default, classifiers, depth_range, "mean")
viz.visualize_comparison(x, SIM_CLASSES_PERC, title="Best 5 Classes Similarity Heatmap")

Same for the best class cases

In [ ]:
viz.visualize_comparison(x, SIM_BEST_CLASS, title="Best Class Similarity Heatmap")